In [70]:
import pandas as pd
df_letters = pd.read_csv('data/all_letters.tsv',sep='\t').drop('Unnamed: 0',axis=1) # remove "drop" if the column is not there

In [71]:
df_commodities = pd.read_csv('data/commodities.tsv',sep='\t').drop('Unnamed: 0',axis=1)

In [72]:
commodities = df_commodities.to_dict(orient='records')

In [73]:
import json

In [74]:
global matching_mode
'''options:
        "exact": "kaneel" and "kaneelstokjes" won't match (minimise false positives)
        "broad": "gin" and "beginnen" will match (minimise false negatives)
        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don't (a middle ground)
'''


'options:\n        "exact": "kaneel" and "kaneelstokjes" won\'t match (minimise false positives)\n        "broad": "gin" and "beginnen" will match (minimise false negatives)\n        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don\'t (a middle ground)\n'

In [75]:
def inside_another_word(text, start, end): # to avoid finding "gin" in "beginnen"
    left = (start > 0) and text[start-1].isalpha()
    right = (end < len(text)-1) and text[end+1].isalpha()
    if matching_mode == 'left_only':
        return left 
    return left or right

def find_label(text, label):
    start = text.find(label)
    end = start + len(label) -1 # last character, same as spacy does
    if start == -1:
        return start
    if matching_mode != 'broad' and inside_another_word(text, start, end):
        return find_label(text[end:],label) # look further in the remaining part of the text
    return start

In [76]:
matching_mode = 'exact'
find_label('We beginnen met een beetje gin', 'gin')

20

In [77]:
def find_commodities(text): 
    res = {}
    if type(text) != str:
        return '{}'
    for row in commodities:
        concept_uri, labels = row['concept_uri'], row['labels']
        for label in labels.split('; '):
            if find_label(text.lower(), label.lower()) != -1: # slow but ok for now
                if concept_uri in res:
                    res[concept_uri].append(label)
                else:
                    res[concept_uri] = [label]

    return json.dumps(res)

In [78]:
from tqdm.notebook import tqdm_notebook # progress bar
tqdm_notebook.pandas()

In [79]:
matching_mode = 'broad'
df_letters['commodities_broad'] = df_letters['text'].progress_map(find_commodities)
matching_mode = 'exact'
df_letters['commodities_precise'] = df_letters['text'].progress_map(find_commodities)
matching_mode = 'left_only'
df_letters['commodities'] = df_letters['text'].progress_map(find_commodities)


  0%|          | 0/607 [00:00<?, ?it/s]

  0%|          | 0/607 [00:00<?, ?it/s]

  0%|          | 0/607 [00:00<?, ?it/s]

In [80]:
df_letters.iloc[3]['commodities_broad']

'{"https://digitaalerfgoed.poolparty.biz/globalise/4a15fa9c-b390-49ea-9b4b-e247c2fb9d72": ["Rye"], "https://digitaalerfgoed.poolparty.biz/globalise/101316fc-b7d7-4460-aa6f-98827bb3bba8": ["Noot"], "https://digitaalerfgoed.poolparty.biz/globalise/8b40ffef-9d82-45d2-8b5b-2554edc6e754": ["nagel", "Kruidnagel"], "https://digitaalerfgoed.poolparty.biz/globalise/a60b5a4c-a015-4a96-a848-1b1022bed2b7": ["Was"], "https://digitaalerfgoed.poolparty.biz/globalise/7d5a258d-2095-4df5-b51f-540f859d5fb5": ["Bier"], "https://digitaalerfgoed.poolparty.biz/globalise/0dbe4d8e-7603-46f4-85bd-3cdd405d0909": ["Gin"], "https://digitaalerfgoed.poolparty.biz/globalise/377eb937-15e9-4b08-9338-48802dba3e5d": ["leech", "lijk"], "https://digitaalerfgoed.poolparty.biz/globalise/71dadcd6-78ae-45ac-811e-b0e4aefc18a0": ["leer"], "https://digitaalerfgoed.poolparty.biz/globalise/192a4e33-b6d0-40df-845c-4b6cfa5ea15b": ["serge"], "https://digitaalerfgoed.poolparty.biz/globalise/c0cd87a4-dd6c-46ee-9dfc-6570a190086f": ["ras"

In [81]:
df_letters.iloc[3]['commodities']

'{"https://digitaalerfgoed.poolparty.biz/globalise/101316fc-b7d7-4460-aa6f-98827bb3bba8": ["Noot"], "https://digitaalerfgoed.poolparty.biz/globalise/8b40ffef-9d82-45d2-8b5b-2554edc6e754": ["Kruidnagel"], "https://digitaalerfgoed.poolparty.biz/globalise/a60b5a4c-a015-4a96-a848-1b1022bed2b7": ["Was"], "https://digitaalerfgoed.poolparty.biz/globalise/71dadcd6-78ae-45ac-811e-b0e4aefc18a0": ["leer"], "https://digitaalerfgoed.poolparty.biz/globalise/192a4e33-b6d0-40df-845c-4b6cfa5ea15b": ["serge"], "https://digitaalerfgoed.poolparty.biz/globalise/c0cd87a4-dd6c-46ee-9dfc-6570a190086f": ["ras"], "https://digitaalerfgoed.poolparty.biz/globalise/6bfebb97-0ce9-44d9-9be7-d2ede6eae05a": ["lont"], "https://digitaalerfgoed.poolparty.biz/globalise/bce8d2b9-f1dd-4d9b-a9f4-c4cdf6776ab0": ["Steen"], "https://digitaalerfgoed.poolparty.biz/globalise/33b9b97c-1c7f-40f3-9b36-b9f84134cded": ["Slave"], "https://digitaalerfgoed.poolparty.biz/globalise/f091c9a4-9d69-474a-ba78-a06e74b5fbf9": ["Kist"], "https://di

In [82]:
df_letters.iloc[3]['commodities_precise']

'{"https://digitaalerfgoed.poolparty.biz/globalise/101316fc-b7d7-4460-aa6f-98827bb3bba8": ["Noot"], "https://digitaalerfgoed.poolparty.biz/globalise/a60b5a4c-a015-4a96-a848-1b1022bed2b7": ["Was"], "https://digitaalerfgoed.poolparty.biz/globalise/bce8d2b9-f1dd-4d9b-a9f4-c4cdf6776ab0": ["Steen"], "https://digitaalerfgoed.poolparty.biz/globalise/33b9b97c-1c7f-40f3-9b36-b9f84134cded": ["Slave"], "https://digitaalerfgoed.poolparty.biz/globalise/ef4fa0f5-8d28-4763-95e6-3b7aa7a0ab30": ["sloep"], "https://digitaalerfgoed.poolparty.biz/globalise/511e4b40-26a7-4da2-b758-a57699c697d0": ["Schip"], "https://digitaalerfgoed.poolparty.biz/globalise/3622a21f-e9e5-4628-9221-99d824d316db": ["Want"], "https://digitaalerfgoed.poolparty.biz/globalise/b89b5e5a-5b47-4f39-bd2c-d43d5ad8484e": ["Mast", "mast"], "https://digitaalerfgoed.poolparty.biz/globalise/0e6e54a9-23ae-49cb-a993-594778509b61": ["wortelen"], "https://digitaalerfgoed.poolparty.biz/globalise/e745f151-189e-4dbe-85be-1799d28fff0d": ["geit"], "ht

In [83]:
text = df_letters.iloc[3]['text']
start = text.lower().find('kruidnagel')
text[start:start+15]

'kruidnagelen in'

In [84]:
df_letters.to_csv('letters_with_commodities.tsv',sep='\t',index=False)

In [85]:
df_letters

,page,title,rawdate,seq,place,year,month,day,author,pid,status,authorFull,text,rest,commodities_broad,commodities_precise,commodities
0,3,Both; aan boord van het Wapen van Amsterdam vo...,25 februari 1610,I,aan boord van het Wapen van Amsterdam voor Ile...,1610.0,2.0,25.0,Both,NaN,NaN,Pieter Both,"«Scheepshericht, vnl. handelend over uitgedeel...",NaN,"{""https://digitaalerfgoed.poolparty.biz/global...",{},"{""https://digitaalerfgoed.poolparty.biz/global..."
1,3,Both; aan boord van het Wapen van Amsterdam li...,6 augustus 1610,II,aan boord van het Wapen van Amsterdam liggende...,1610.0,8.0,6.0,Both,NaN,NaN,Pieter Both,"«„Journaelsgewijse"" reisbeschrijving »",NaN,"{""https://digitaalerfgoed.poolparty.biz/global...",{},{}
2,3,Both; aan boord van het Wapen van Amsterdam te...,14 oktober 1610,III,aan boord van het Wapen van Amsterdam ter Rede...,1610.0,10.0,14.0,Both,NaN,NaN,Pieter Both,«De inhoud van deze brief is evenmin als die v...,NaN,{},{},{}
3,4,"Both; Kasteel Nassau op Banda-Neira, 18 juli 1611",18 juli 1611,IV,Kasteel Nassau op Banda-Neira,1611.0,7.0,18.0,Both,NaN,NaN,Pieter Both,. . . Principaelijck besich geweest in ’t opbo...,NaN,"{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global..."
4,7,"Both; aan boord van de Vere voor Maleyo, 31 ma...",31 maart 1612,V,aan boord van de Vere voor Maleyo,1612.0,3.0,31.0,Both,NaN,NaN,Pieter Both,"Mijn advys soude oock sijn, dat U E geen licht...",NaN,"{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,745,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",31 december 1765,XIV,Batavia,1765.0,12.0,31.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{},{}
603,883,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",8 april 1766,XV,Batavia,1766.0,4.0,8.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",Nadat het tweede gezandschap op Colombo aangek...,NaN,"{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global...","{""https://digitaalerfgoed.poolparty.biz/global..."
604,893,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",21 oktober 1766,XVI,Batavia,1766.0,10.0,21.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{},{}
605,943,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",31 december 1766,XVII,Batavia,1766.0,12.0,31.0,"Van der Parra, Van Riemsdijk, Schreuder, Van B...",NaN,NaN,"Petrus Albertus van der Parra, Jeremias van Ri...",NaN,NaN,{},{},{}
